In [1]:
import json
import geopandas as gp
from shapely.geometry import Point, LineString, Polygon

In [2]:
# these are the unedited data from Ollie O'Brien's vis repo (https://github.com/oobrien/vis)
with open("static/tfl_lines.geojson", 'r') as f:            
    gj = json.loads(f.read()) 

In [3]:
# Some segments belong to multiple lines
# We need to build new LineStrings with each of these
# this results in a small amount of duplication but that's OK
new_features = []
for feature in gj['features']:
    num_lines = len(feature['properties']['lines'])
    if num_lines > 1:
        for segment in feature['properties']['lines'][1:]:
            new_feature = feature
            new_feature['properties']['line_name'] = segment['name']
            new_feature['properties']['shared'] = True
            new_features.append(
                new_feature
            )
    else:
        feature['properties']['shared'] = False

In [4]:
# assign a new line_name property to existing features
for feature in gj['features']:                              
    feature['properties']['line_name'] = feature['properties']['lines'][0]['name']

In [5]:
# combine existing and new features
gj['features'] = gj['features'] + new_features

In [6]:
# remove 'lines' property, since we don't need it any more
for feature in gj['features']:
    try:
        del feature['properties']['lines']
    except KeyError:
        pass

In [7]:
with open("static/tfl_lines_edited.geojson", 'w') as g:
    json.dump(gj, g)

In [8]:
df = gp.read_file("static/tfl_lines_edited.geojson")

In [9]:
df.head()

,id,shared,line_name,geometry
0,LeaValleyLine1,False,London Overground,"LINESTRING (-0.05941 51.52391, -0.05910 51.523..."
1,LeaValleyLine2,False,London Overground,"LINESTRING (-0.05760 51.53245, -0.05767 51.532..."
2,LeaValleyLine3,False,London Overground,"LINESTRING (-0.05785 51.54118, -0.05780 51.541..."
3,LeaValleyLine4,False,London Overground,"LINESTRING (-0.06116 51.54909, -0.06122 51.549..."
4,LeaValleyLine5,False,London Overground,"LINESTRING (-0.06845 51.55866, -0.06868 51.559..."


In [10]:
# Mapbox GL can only extrude polygons.
# LineStrings can be converted into Polygons by buffering them
# choose an appropriate width here
df.geometry = df.geometry.buffer(0.000100)
# simplify polygons a little bit. Threshold may need to be fine-tuned
df.geometry = df.geometry.simplify(0.0000001)

/var/folders/wy/kj5lv5hx1g78fzmjjm64q4340000gn/T/ipykernel_3285/3974467899.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df.geometry = df.geometry.buffer(0.000100)


In [12]:
# fix up other unfamiliar lines
df['line_name'] = df['line_name'].replace('East London', 'London Overground')
df['line_name'] = df['line_name'].replace('Crossrail 2', 'Crossrail')
df['line_name'] = df['line_name'].replace('TfL Rail', 'Crossrail')

In [13]:
df.line_name.unique()

array(['London Overground', 'Victoria', 'Piccadilly', 'District',
       'Circle', 'Hammersmith & City', 'Metropolitan', 'Central',
       'Jubilee', 'DLR', 'Bakerloo', 'Northern', 'Waterloo & City',
       'Elizabeth line', 'Tramlink', 'Crossrail', 'IFS Cloud Cable Car',
       'Thameslink 6tph line'], dtype=object)

In [14]:
df.to_file("static/tube_polygons.geojson", driver="GeoJSON")

- Shell out to: `ogr2ogr -f "GeoJSON" -lco COORDINATE_PRECISION=7 static/tube_polygons_trimmed.geojson static/tube_polygons.geojson`
- `rm static/tube_polygons.geojson`
- `mv static/tube_polygons_trimmed.geojson static/tube_polygons.geojson`